In [534]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn_pandas import gen_features
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report , confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score



In [535]:
df = pd.read_csv("chocolate_bars.csv")

In [536]:
df.head(10)

,id,manufacturer,company_location,year_reviewed,bean_origin,bar_name,cocoa_percent,num_ingredients,ingredients,review,rating
0,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.0,"B,S,C","rich cocoa, fatty, bready",3.25
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.0,"B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.0,"B,S,C","cocoa, blackberry, full body",3.75
3,2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68.0,3.0,"B,S,C","chewy, off, rubbery",3.00
4,2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72.0,3.0,"B,S,C","fatty, earthy, moss, nutty,chalky",3.00
5,2546,5150,U.S.A.,2021,Uganda,"Semuliki Forest, batch 1",80.0,3.0,"B,S,C","mildly bitter, basic cocoa, fatty",3.25
6,2542,5150,U.S.A.,2021,India,"Anamalai, batch 1",68.0,3.0,"B,S,C","milk brownie, macadamia,chewy",3.50
7,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,4.0,"B,S,C,L","vegetal, nutty",3.50
8,797,A. Morin,France,2012,Peru,Peru,63.0,4.0,"B,S,C,L","fruity, melon, roasty",3.75
9,1011,A. Morin,France,2013,Panama,Panama,70.0,4.0,"B,S,C,L","brief fruit note, earthy, nutty",2.75


In [537]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2530 entries, 0 to 2529
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                2530 non-null   int64  
 1   manufacturer      2530 non-null   object 
 2   company_location  2530 non-null   object 
 3   year_reviewed     2530 non-null   int64  
 4   bean_origin       2530 non-null   object 
 5   bar_name          2530 non-null   object 
 6   cocoa_percent     2530 non-null   float64
 7   num_ingredients   2443 non-null   float64
 8   ingredients       2443 non-null   object 
 9   review            2530 non-null   object 
 10  rating            2530 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 217.6+ KB


In [538]:
df.describe()

,id,year_reviewed,cocoa_percent,num_ingredients,rating
count,2530.000000,2530.000000,2530.000000,2443.000000,2530.000000
mean,1429.800791,2014.374308,71.639723,3.041343,3.196344
std,757.648556,3.968267,5.616724,0.913728,0.445321
min,5.000000,2006.000000,42.000000,1.000000,1.000000
25%,802.000000,2012.000000,70.000000,2.000000,3.000000
50%,1454.000000,2015.000000,70.000000,3.000000,3.250000
75%,2079.000000,2018.000000,74.000000,4.000000,3.500000
max,2712.000000,2021.000000,100.000000,6.000000,4.000000


In [539]:
df.isna().sum()

id                   0
manufacturer         0
company_location     0
year_reviewed        0
bean_origin          0
bar_name             0
cocoa_percent        0
num_ingredients     87
ingredients         87
review               0
rating               0
dtype: int64

In [540]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f'Унікальні значення для стовбця "{column}": {unique_values}')

Унікальні значення для стовбця "id": [2454 2458 2542 2546  797 1011 1015 1019 1315 1319 1676 1680 1704 1876
 2206 2648 1462 1470 2462 2470  705 2438 2442  370  316  502  508  636
 1061 1173 1215 1992 1944 1125 1133 1129 1732 1728 2044  147  129  175
  304  363  544  470  725  327  464  322 1964 1145 1494 1498   75  123
  170  979 2088 2092 2434  572 1065 1259 1852 2586 1379 1375 1602 1534
 1598 1904 1928 1724 1900 1908 1924 2020 2028 2024 2068 2254 2450 2330
 2166 2162  300  355  486  600  531  745  729  947 1193 1181 2562 2566
  141  331  647  661 1780 2056 2672  999  995 1474 2146 1454 2290 2294
  983 1295 1554 1980  955 1880 1840 1868 2374 1948 1784 1788  586 1804
 1800 1864 2190 1768 2246  773  757 1141 1482 1486 2084  233  256  414
  423  431  558  565  478  963 2108 2114 2422 2574 1331 2590 1046  911
 1752 1756 1740 1996   81   24   32   48  199  336  395  761  629  672
 1042 1038 1418 1339 1912 2250 2554  341 1267 1271 2194 2096 1255 1355
 1984 1518 1514 1149 1235 1231 1638 2052

B (Beans), S (Sugar), S* (Sweetener other than sugar or beet sugar), C (Cocoa Butter), (V) Vanilla, (L) Lecithin, Sa (Salt)

In [541]:
Ingredients = {
    "Beans": "B",
    "Sugar": "S",
    "Sweetener": "S*",
    "Cocoa Butter": "C",
    "Vanilla": "V",
    "Lecithin": "L",
    "Salt": "Sa"
}

for key in Ingredients:
    df[key] = np.where(df['ingredients'].str.contains(Ingredients[key],case=True), 1, 0)

df.head() 

,id,manufacturer,company_location,year_reviewed,bean_origin,bar_name,cocoa_percent,num_ingredients,ingredients,review,rating,Beans,Sugar,Sweetener,Cocoa Butter,Vanilla,Lecithin,Salt
0,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.0,"B,S,C","rich cocoa, fatty, bready",3.25,1,1,1,1,0,0,0
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.0,"B,S,C","cocoa, vegetal, savory",3.50,1,1,1,1,0,0,0
2,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.0,"B,S,C","cocoa, blackberry, full body",3.75,1,1,1,1,0,0,0
3,2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68.0,3.0,"B,S,C","chewy, off, rubbery",3.00,1,1,1,1,0,0,0
4,2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72.0,3.0,"B,S,C","fatty, earthy, moss, nutty,chalky",3.00,1,1,1,1,0,0,0


In [542]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f'Унікальні значення для стовбця "{column}": {unique_values}')

Унікальні значення для стовбця "id": [2454 2458 2542 2546  797 1011 1015 1019 1315 1319 1676 1680 1704 1876
 2206 2648 1462 1470 2462 2470  705 2438 2442  370  316  502  508  636
 1061 1173 1215 1992 1944 1125 1133 1129 1732 1728 2044  147  129  175
  304  363  544  470  725  327  464  322 1964 1145 1494 1498   75  123
  170  979 2088 2092 2434  572 1065 1259 1852 2586 1379 1375 1602 1534
 1598 1904 1928 1724 1900 1908 1924 2020 2028 2024 2068 2254 2450 2330
 2166 2162  300  355  486  600  531  745  729  947 1193 1181 2562 2566
  141  331  647  661 1780 2056 2672  999  995 1474 2146 1454 2290 2294
  983 1295 1554 1980  955 1880 1840 1868 2374 1948 1784 1788  586 1804
 1800 1864 2190 1768 2246  773  757 1141 1482 1486 2084  233  256  414
  423  431  558  565  478  963 2108 2114 2422 2574 1331 2590 1046  911
 1752 1756 1740 1996   81   24   32   48  199  336  395  761  629  672
 1042 1038 1418 1339 1912 2250 2554  341 1267 1271 2194 2096 1255 1355
 1984 1518 1514 1149 1235 1231 1638 2052

In [543]:
df.isna().sum()

id                   0
manufacturer         0
company_location     0
year_reviewed        0
bean_origin          0
bar_name             0
cocoa_percent        0
num_ingredients     87
ingredients         87
review               0
rating               0
Beans                0
Sugar                0
Sweetener            0
Cocoa Butter         0
Vanilla              0
Lecithin             0
Salt                 0
dtype: int64

In [544]:
asia = ['Vietnam', 'Indonesia', 'Sri Lanka', 'Malaysia', 'Philippines', 'India', 'Taiwan', 'Thailand', 'Sumatra', 'Sulawesi']
africa = ['Tanzania', 'Madagascar', 'Fiji', 'Uganda', 'Burma', 'Ghana', 'Nigeria', 'Liberia', 'Sierra Leone', 'Ivory Coast', 'Gabon', 'Sao Tome', 'Sao Tome & Principe', 'Cameroon', 'Sao Tome & Principe', 'DR Congo']
america = ['Dominican Republic', 'Venezuela', 'Peru', 'Bolivia', 'Panama', 'Colombia', 'Brazil', 'Ecuador', 'Cuba', 'Mexico', 'Nicaragua', 'Belize', 'Jamaica', 'Grenada', 'Guatemala', 'Honduras', 'Costa Rica', 'Haiti', 'Puerto Rico', 'Martinique', 'St. Lucia', 'St.Vincent-Grenadines', 'El Salvador', 'Suriname', 'DR Congo']
oceania = ['Vanuatu', 'Papua New Guinea', 'Solomon Islands', 'Samoa', 'Australia']

In [545]:
df['asia'] = df['bean_origin'].apply(lambda x: 1 if x in asia else 0)
df['africa'] =  df['bean_origin'].apply(lambda x: 1 if x in africa else 0)
df['america'] =  df['bean_origin'].apply(lambda x: 1 if x in america else 0)
df['oceania'] =  df['bean_origin'].apply(lambda x: 1 if x in oceania else 0)

df['dark_choc'] = df['cocoa_percent'].apply(lambda x: 1 if x > 60 else 0)
df['milk_choc'] = df['cocoa_percent'].apply(lambda x: 1 if x <= 60 else 0)

In [546]:
df['rating'].value_counts()


rating
3.50    565
3.00    523
3.25    464
2.75    333
3.75    300
2.50    166
4.00    112
2.00     33
2.25     17
1.50     10
1.00      4
1.75      3
Name: count, dtype: int64

In [554]:
df['rating_category'] = pd.cut(df['rating'], [1.9, 2.9, 3.49, 3.9, 5.0], labels=False)
# 1 - low
# 2 - med
# 3 - high

"""
df['rating_category'] = np.select(
    [df['rating'].between(1, 2.50), df['rating'].between(2.75, 3.25), df['rating'].between(3.5, 4)],
    ['1', '2', '3'],
    default='Unknown'
)
"""

''

In [548]:
df['rating_category'].value_counts()


rating_category
1.0    987
2.0    865
0.0    549
3.0    112
Name: count, dtype: int64

In [549]:
df.head()

,id,manufacturer,company_location,year_reviewed,bean_origin,bar_name,cocoa_percent,num_ingredients,ingredients,review,...,Vanilla,Lecithin,Salt,asia,africa,america,oceania,dark_choc,milk_choc,rating_category
0,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.0,"B,S,C","rich cocoa, fatty, bready",...,0,0,0,0,1,0,0,1,0,1.0
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.0,"B,S,C","cocoa, vegetal, savory",...,0,0,0,0,0,1,0,1,0,2.0
2,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.0,"B,S,C","cocoa, blackberry, full body",...,0,0,0,0,1,0,0,1,0,2.0
3,2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68.0,3.0,"B,S,C","chewy, off, rubbery",...,0,0,0,0,1,0,0,1,0,1.0
4,2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72.0,3.0,"B,S,C","fatty, earthy, moss, nutty,chalky",...,0,0,0,0,0,1,0,1,0,1.0


In [550]:
X = df.drop(['rating','rating_category', 'manufacturer', 'id', 'bar_name', 'company_location', 'ingredients', 'bean_origin', 'review', 'year_reviewed'], axis=1)
y = df['rating_category']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_features = gen_features(
    columns=[[col] for col in df.drop(['rating','rating_category', 'manufacturer', 'id', 'bar_name', 'company_location', 'ingredients', 'bean_origin', 'review', 'year_reviewed'], axis=1)[['cocoa_percent','num_ingredients']].columns],
    classes=[
        {'class': SimpleImputer, 'strategy': 'most_frequent'},
        {'class': StandardScaler}
    ]
)
categorical_features = gen_features(
    columns=[[col] for col in df.drop(['rating','rating_category', 'manufacturer', 'id', 'bar_name', 'company_location', 'ingredients', 'bean_origin', 'review', 'year_reviewed'], axis=1).columns if col not in ['cocoa_percent', 'num_ingredients']]
)

mapper = DataFrameMapper(
  numerical_features + categorical_features, df_out= True, default=False)

X_train_prepr = mapper.fit_transform(X_train)
X_test_prepr = mapper.transform(X_test)

In [551]:
X_train_prepr

,cocoa_percent,num_ingredients,Beans,Sugar,Sweetener,Cocoa Butter,Vanilla,Lecithin,Salt,asia,africa,america,oceania,dark_choc,milk_choc
1990,-0.305484,-1.150873,1,1,1,0,0,0,0,0,0,0,0,1,0
157,0.051399,-0.031523,1,1,1,1,0,0,0,0,0,0,0,1,0
1472,-0.305484,-1.150873,1,1,1,0,0,0,0,0,0,1,0,1,0
790,0.943607,-0.031523,1,1,1,1,0,0,0,0,1,0,0,1,0
2075,-0.305484,-0.031523,1,1,1,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,-0.305484,-1.150873,1,1,1,0,0,0,0,0,0,1,0,1,0
1095,3.441788,2.207177,1,1,1,1,1,1,0,0,0,0,0,1,0
1130,-1.376133,-0.031523,1,1,1,0,0,1,0,0,0,1,0,1,0
1294,0.051399,-1.150873,1,1,1,0,0,0,0,0,0,1,0,1,0


In [552]:
y_train

1990    1.0
157     2.0
1472    2.0
790     0.0
2075    1.0
       ... 
1638    1.0
1095    1.0
1130    0.0
1294    0.0
860     1.0
Name: rating_category, Length: 2024, dtype: float64

# SGDClassifier

In [553]:
sgd = SGDClassifier()

params = {
    'alpha': [0.0001, 0.001, 0.01],
    'penalty': ['l1', 'l2'],
    'loss': ['hinge', 'modified_huber']
}

grid_search_sgd = GridSearchCV(sgd, params, cv=5)
grid_search_sgd.fit(X_train_prepr, y_train)

print("Найкращі параметри:", grid_search_sgd.best_params_)

G:\Ilya\ML\venv\Lib\site-packages\sklearn\utils\_array_api.py:290: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y contains NaN.

In [ ]:
y_pred_sgd = grid_search_sgd.predict(X_test_prepr)

# Обчислення метрик
accuracy_sgd = accuracy_score(y_test, y_pred_sgd)
precision_sgd = precision_score(y_test, y_pred_sgd, average='weighted')
recall_sgd = recall_score(y_test, y_pred_sgd, average='weighted')
f1_sgd = f1_score(y_test, y_pred_sgd, average='weighted')

print("Accuracy:", accuracy_sgd)
print("Precision:", precision_sgd)
print("Recall:", recall_sgd)
print("F1 Score:", f1_sgd)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_sgd))


# KNNClassifier

In [ ]:
knn = KNeighborsClassifier()

params = {'n_neighbors': [3, 5, 7, 9], 'p': [1, 2]}

grid_search = GridSearchCV(knn, params, cv=5)
grid_search.fit(X_train_prepr, y_train)

# Вивід найкращих параметрів
print("Найкращі параметри:", grid_search.best_params_)

best_knn = grid_search.best_estimator_
y_pred_knn = best_knn.predict(X_test_prepr)


In [ ]:
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy)

precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_knn, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("Weighted F1:", f1)

class_report = classification_report(y_test, y_pred_knn)
print("Classification Report:")
print(class_report)

Побудувати матриці невідповідностей (confusion matrix [API: 3]), проаналізувати матриці;

In [ ]:
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)

conf_matrix_normalized_recall_knn = confusion_matrix(y_test, y_pred_knn, normalize='true')
conf_matrix_normalized_precision_knn = confusion_matrix(y_test, y_pred_knn, normalize='pred')


fig, axes = plt.subplots(1, 3, figsize=(15, 6))
sns.heatmap(conf_matrix_knn, annot=True, fmt="4.0f", cmap="crest", ax=axes[0])
axes[0].set_title("Confusion Matrix")
#по recall
sns.heatmap(conf_matrix_normalized_recall_knn, annot=True, fmt=".2f", cmap="crest",ax=axes[1])
axes[1].set_title("Confusion Matrix NO Recall")
#по precision
sns.heatmap(conf_matrix_normalized_precision_knn, annot=True, fmt=".2f", cmap="crest",ax=axes[2])
axes[2].set_title("Confusion Matrix NO Precision")
plt.tight_layout()
plt.show()

print("Confusion Matrix (KNN):\n", conf_matrix_knn)
print("\nConfusion Matrix (Normalized - Recall):\n", conf_matrix_normalized_recall_knn)
print("\nConfusion Matrix (Normalized - Precision):\n", conf_matrix_normalized_precision_knn)

# DecisionTreeClassifier: